# Advanced Geocomputing Final Project

## Customized Personal Route Creation Tool

#### By Laure Briol with the assistance of Generative AI

My final project for Advanced Geocomputing (GEOG 5543) aimed to create a personalized routing service. This routing service enabled a user to visit all locations, while being able to avoid designated areas on a map and while minimizing their travel time. The project's initial scope focused on how to improve a shopping trip/experience. Using the tool, a user can go to multiple store locations, without backtracking on their drive. A user can choose multiple store location's and then be given the optimal route to reach each destination. There are some areas a user may want to avoid and the personal route creation tool takes these preferences into account, in order to improve the overall shopping experience. Therefore, the route creation tool implemented a feature, which allowed for a user to draw polygon areas that the routing service will avoid, when planning the route.

## Implementation

It was a goal to try out alternative services other than the expensive Google API’s, when implementing this project. To do this, there is a website called https://openrouteservice.org/, which has an open source API that is able to provide routing services for free. An added benefit of this service is that a user can state which locations they want to avoid and the API will automatically avoid them when creating the best route.

## Code Sections

The project is an interactive map, meaning it cannot split into multiple pieces. Instead, the code is organized into these sections:

### Section 1: Global Variables:

* This section contains all the code that is repetitively called in the program.
* This includes setting up the API key, colors for the map markers, lists to store the locations of the stops, and setting up an empty default map. 

### Section 2: Avoid area polygons

* This section contains code designed for prepping the 'avoiding areas polygon' feature.
* This section adds in the map control button to draw a polygon. When the polygon is drawn, the polygon is added to a list of locations and is displayed on the map. 

### Section 3: Adding locations

* This section contains code designed for adding new locations onto the map.
* This section adds in the button to enable adding locations and also adds the button to turn on ‘add new locations’.
* Also included in this section is the checkbox option to ‘return to start’.

### Section 4: Saving/loading map data

* This section contains code designed for saving data from the map and then loading said data onto the map.
* This section adds the buttons and text box to save data and load data. Additionally, this section implements the feature to save the data to a json file when the save button is clicked.

### Section 5: Optimizing route

* This is the largest section of the code. This section creates an optimized route for the person to follow.
* This section is run when the user clicks the ‘run optimization’ button.
* The function starts by running a function called get_time_matrix_with_avoid to create a ‘time matrix’, calculating the time it takes to go between all point pair permutations.
* OR tools is used to optimize the route between the two locations. This required the data to be re-adjusted to its own special format, which ChatGPT shared how to do. Then, ‘parameters’/settings were enabled for doing the optimization algorithms.
* Once the OR tools finds the best route between all the locations, the program then updates the colors of markers to better indicate the order in which a person is traveling.
* The openrouteservice API is also called to give the proper step by step directions for each ‘leg’ of the journey between two points.
    * The program keeps track of the total amount of time traveled, based on the step by step travel times and travel distances.
    * The API also shares the polyline for each leg, so that is displayed on the map as well.
* The program finally prints out the step by step directions and total travel times at the bottom of the screen.


### Output

The output is a final interactive map that can be used to create a custom route. There are buttons and a text input box at the top for saving & loading existing map json features. Below that, there is a button to toggle, which enables the user to add new points to the map and a checkbox to enable a user to return to the start point. The map itself is then displayed and has the option to draw a polygon area for avoiding features (when the 'Add Locations' is toggled to off). An 'Optimize Routes' button is displayed at the bottom of the screen. When the optimize routes button is clicked, the program will find the best route and display it on the map, as well as print step-by-step directions below the map display.

In [4]:
#!pip install openrouteservice
#!pip install ortools

In [5]:
#Import statements
#ipywidgets is to add buttons and check boxes
import ipywidgets as widgets
#this is a bunch of mapping features for ipyleaflet
from ipyleaflet import Map, Marker, Polygon, basemaps, DrawControl, LayerGroup, AwesomeIcon, Polyline
#this is updating the output of the jupyter notebook without filling a bunch of space
from IPython.display import display, clear_output
#openrouteservice is the open source routing API, it's like google maps but open source
import openrouteservice
#time is used to pause the application so it doesn't call the API too fast
import time
#OR tools -- operations research tools, this is used to solve and find the fastest route between all the points
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
#shapely geometry is used to help create the polygon that we avoid areas from
from shapely.geometry import mapping, shape
#unary union combines polyline results into one line
from shapely.ops import unary_union
#import json for saving and loading markers and avoided areas
import json

In [6]:
#section 1: global variables

#openrouteservice API key, this is the alternative to Google, you can get the API key here
# https://openrouteservice.org/
api_key = 'INSERT_HERE'  # Replace with your own API key
#we start talking to the openrouteservice API using the python package
ors_client = openrouteservice.Client(key=api_key)

#convert from meters to miles when measuring the final route because that's easier to interporet
MILES_PER_METER = 1 / 1609.34

#when we create a custom route, we want to make the stop colors in this order so we know which stop is which on the map
marker_colors = [
    'blue', 'purple', 'green', 'gray', 'orange', 'black',
    'lightblue', 'lightgreen', 'pink', 'beige', 'lightgray', 'darkblue',
    'darkgreen', 'darkpurple', 'white', 'cadetblue', 'red', 'darkred', 'lightred'
]

#these are lists to store different locations, marker colors, and areas to avoid for our final map
#this is the list of locaitons we want to visit
locations = []
#this is the list of the map markers/icons for our locations
location_markers = []
#this is a list of the areas that we want to avoid on the map
avoid_areas = []

#map centered on the Twin Cities
center = [44.96, -93.18]
#creat the map 'object' that displays using openstreetmap basemap
m = Map(center=center, zoom=11, basemap=basemaps.OpenStreetMap.Mapnik)

#create the layers for each type of thing we're going to display
#marker for stops that we will visit
marker_layer = LayerGroup()
#route polyline for final route display
route_layer = LayerGroup()
#polygon for areas that we want to avoid going to
avoid_area_layer = LayerGroup()
#link the marker layer, route layer, and avoid areas layer to the map m so they will display
m.add_layer(marker_layer)
m.add_layer(route_layer)
m.add_layer(avoid_area_layer)

#Section 2: Avoid Areas Polygon

#create the option to be able to draw polygons on the map for our avoid areas
draw_control = DrawControl(polyline={}, circlemarker={}, circle={}, rectangle={})

#this is the style/symbology for our avoid areas polygons while we are drawing it
draw_control.polygon = {
    "shapeOptions": {
        "color": "#0000FF",  #blue color for the border
        "fillColor": "#0000FF",  #blue color for the fill
        "fillOpacity": 0.5 #fill is 50% transparency
    },
    "drawError": {
        "color": "#0000FF", #if it doesn't draw correcntly have it error
        "message": "Error drawing shape!"
    },
    "allowIntersection": False
}

#function called handle_draw, which is used to keep track of drawing polygons on the map
#and save the ones that are finished being drawn, then add the completed drawing to the map
def handle_draw(self, action, geo_json):
    #when a user finishes drawing a polygon, it is 'created'
    if action == 'created':
        #make sure the thing they drew is a polygon
        if geo_json['geometry']['type'] == 'Polygon':
            #add the polygon to the list storing the information about all the areas to avoid
            avoid_areas.append(geo_json['geometry'])
            #create the polygon that appears on the map
            polygon = Polygon(
                locations=[(lat, lng) for lng, lat in geo_json['geometry']['coordinates'][0]],
                color="blue",
                fill_color="blue",
                fill_opacity=0.5
            )
            #add the polygon to the map
            avoid_area_layer.add_layer(polygon)

#when the user starts drawing a polygon on the map, keep track of what is going on
draw_control.on_draw(handle_draw)


#Section 3: Adding locaitons

#when the user 'clicks' on the map, we figure out if a 'stop' is going to be added
def on_map_click(**kwargs):
    #if we have a setting turned on to 'add locations', we continue to check if the user clicked on an icon
    if add_locations_mode.value:
        #if the user 'clicked' and we know that they are trying to add a location, then add a stop location to the map
        if kwargs.get('type') == 'click':
            #get the location of the marker on the map
            latlng = kwargs.get('coordinates')
            #keep track of the latitude and longitude
            lat, lng = latlng[0], latlng[1]
            #figure out what 'index' in the list of locations we are at. This is used to know what color and symbol to make the marker
            idx = len(locations)
            #set the very first marker to have a 'home' symbol in 'pink' color.
            if idx == 0:
                #create an icon with a home symbol and pink color
                icon = AwesomeIcon(name='home', marker_color='pink', icon_color='white')
            #every subsequent marker is now 'blue' with a 'shopping bag' symbol.
            else:
                #create an icon with a shopping bag symbol and blue color
                icon = AwesomeIcon(name='shopping-bag', marker_color='blue', icon_color='white')
            #create a 'marker' object
            marker = Marker(location=(lat, lng), icon=icon, draggable=False)
            #add the marker to our map
            marker_layer.add_layer(marker)
            #add the stop location to the list of locations
            locations.append({'name': f'Location {idx}', 'lat': lat, 'lng': lng})
            #add the marker to the list of markers, this is so we can change the color later and add back into the map
            location_markers.append(marker)

#when the user interacts with the map, keep track of what is going on (we want to check for when they 'click'
m.on_interaction(on_map_click)

#this is the button itself to allow us to add new stop locations to the map
add_locations_mode = widgets.ToggleButton(
    value=False,
    description='Add Locations',
    button_style='info',
    tooltip='Toggle to add locations by clicking on the map'
)

#when the button is clicked, change what the icon looks like
def toggle_mode(change):
    #when the user clicks the button to turn on adding locations (change['new'] reports as True)
    if change['new']:
        #change what the button says
        add_locations_mode.description = 'Adding Locations'
        #change the color of the button
        add_locations_mode.button_style='success'
        #disable drawing when adding locations
        if draw_control in m.controls:
            m.remove_control(draw_control)
    #if the user click the button to turn off adding locations
    else:
        add_locations_mode.description = 'Add Locations'
        add_locations_mode.button_style='info'
        #enable drawing when not adding locations
        if draw_control not in m.controls:
            m.add_control(draw_control)

#keep track of when someone clicks the button, when it is clicked call the function above
add_locations_mode.observe(toggle_mode, 'value')

#checkbox for "Return to Start" option
return_to_start_checkbox = widgets.Checkbox(
    value=False,
    description='Return to Start',
    disabled=False,
    indent=False
)

#Section 4: Saving/Loading map data

#create Save button
save_button = widgets.Button(
    description='Save Map Locations',
    button_style='info'
)

#create load button
load_button = widgets.Button(
    description='Load Map Locations',
    button_style='info'
)

#create text box input
file_name_widget = widgets.Text(
    value='markers_and_areas.json',
    description='File Name:',
    layout=widgets.Layout(width='300px')
)

#when the save button is clicked, this function is run to handle saving locations and avoid areas from the map
def save_markers_and_areas(button):
    #pick out the data we want to save
    data_to_save = {
        'locations': locations,
        'avoid_areas': avoid_areas
    }
    #get the file name from the text box
    file_name = file_name_widget.value
    try:
        #open the file to write
        with open(file_name, 'w') as f:
            #save the data to the file
            json.dump(data_to_save, f)
        print(f"Markers and avoid areas saved to {file_name}.")
    #if there is an error, report it
    except Exception as e:
        print(f"Error saving to {file_name}: {e}")

#when the load button is clicked, this function is run to handle loading locations and avoid areas to the map
def load_markers_and_areas(button):
    #get the file name from the text box
    file_name = file_name_widget.value
    try:
        #open the file to get the data
        with open(file_name, 'r') as f:
            data_loaded = json.load(f)
        #append new locations to existing ones
        loaded_locations = data_loaded.get('locations', [])
        #add on to the existing list of data
        for loc in loaded_locations:
            #index, where in the list to save the data (the end of it)
            idx = len(locations)
            #latitude and longitude
            lat = loc['lat']
            lng = loc['lng']
            #if this is the first location, make it pink with a home icon
            if idx == 0:
                icon = AwesomeIcon(name='home', marker_color='pink', icon_color='white')
            #if it isn't the first location, make it blue with a shopping bag icon
            else:
                icon = AwesomeIcon(name='shopping-bag', marker_color='blue', icon_color='white')
            #create a new marker object
            marker = Marker(location=(lat, lng), icon=icon)
            #add the marker to the map layer
            marker_layer.add_layer(marker)
            #add the information about the marker to our lists
            locations.append({'name': f'Location {idx}', 'lat': lat, 'lng': lng})
            location_markers.append(marker)
        #add in new avoid areas to existing ones
        loaded_avoid_areas = data_loaded.get('avoid_areas', [])
        #load the polygons of new avoid_areas
        for area in loaded_avoid_areas:
            #add the polygon to our list
            avoid_areas.append(area)
            #create a polygon object to put on the map
            polygon = Polygon(
                locations=[(lat, lng) for lng, lat in area['coordinates'][0]],
                color="blue",
                fill_color="blue",
                fill_opacity=0.5
            )
            #add the area to the map
            avoid_area_layer.add_layer(polygon)
        #if everything worked, print status
        print(f"Markers and avoid areas loaded from {file_name}.")
    #if something didn't work, print status
    except Exception as e:
        print(f"Error loading from {file_name}: {e}")

#when the buttons are clicked, call the function to handle what happens when it is clicked
save_button.on_click(save_markers_and_areas)
load_button.on_click(load_markers_and_areas)

#display the buttons and file name input
display(widgets.HBox([save_button, load_button, file_name_widget]))

#then, display the add location button and return to start
display(widgets.HBox([add_locations_mode, return_to_start_checkbox]))
#then, display the map itself
display(m)

#button to run optimization

#create a button widget that users can click to start optimizing their route
optimize_button = widgets.Button(
    description='Optimize Route',  # set the text displayed on the button
    button_style='success'         # set the style/color of the button to indicate a positive action
)


#section 5: optimizing route

#create an output widget area at the bottom of the screen to display results and directions
output = widgets.Output()

#function to handle route optimization when the optimize button is clicked
def optimize_route(button):
    #start displaying text in the 'output' widget
    with output:
        #clear any existing text in the output widget
        clear_output()
        print("starting route optimization...")  # inform the user that optimization has started
        #check if there are at least two locations to create a route
        if len(locations) < 2:
            print("please select at least two locations.")  # prompt the user to add more locations
            return  # exit the function if there are not enough locations
        
        #clear any existing routes from the previous optimization
        route_layer.clear_layers()
        
        #check if there are any areas to avoid
        if avoid_areas:
            #create shapely polygons from the avoid areas data
            avoid_polygons = [shape(area) for area in avoid_areas]
            #combine all avoid polygons into a single multipolygon for easier processing
            avoid_polygon = unary_union(avoid_polygons)
        else:
            #if there are no avoid areas, set avoid_polygon to none
            avoid_polygon = None
        
        #calculate the number of API calls needed based on the number of locations
        #get the total number of locations
        num_locations = len(locations)
        #calculate total possible pairs of locations
        num_time_matrix_calls = num_locations * (num_locations - 1)
        #estimate total time for API calls in seconds, assuming 1.5 seconds per API call
        time_estimate_matrix = int(num_time_matrix_calls * 1.5)
        print(f"preparing to make {num_time_matrix_calls} api calls to build the time matrix (approximately {time_estimate_matrix} seconds).")
        
        try:
            #generate the time matrix (finding all possible permutations of locations)
            time_matrix = get_time_matrix_with_avoid(locations, ors_client, avoid_polygon)
        except Exception as e:
            #print error if it happens
            print(f"error fetching time matrix: {e}")
            return
        
        #this is creating a 'data model' which is like a standardized data format (like a spreadsheet)
        #to use with the OR tools route optimizer 
        data = create_data_model(time_matrix)
        
        try:
            #if the return to start checkbox is on, we want to return to start
            if return_to_start_checkbox.value:
                #in the standardized data model, the 'depot' is the starting location
                #we want to start from the starting locaiton we picked
                data['depot'] = 0
                #then, we again use the standard format to create a 'manager' to pick out the route for us
                #we have the time matrix, only one vehicle, the starting location, and the ending location (which is the same as the start)
                manager = pywrapcp.RoutingIndexManager(
                    len(data['time_matrix']), data['num_vehicles'], [data['depot']], [data['depot']]
                )
            #if the user does not want to return to the starting location
            else:
                #in the standardized data model, the 'depot' is the starting location
                #we want to start from the starting locaiton we picked
                data['depot'] = 0
                #then, we again use the standard format to create a 'manager' to pick out the route for us
                #we have the time matrix, only one vehicle, the starting location, and no ending location
                manager = pywrapcp.RoutingIndexManager(
                    len(data['time_matrix']), data['num_vehicles'], data['depot']
                )
        except Exception as e:
            #if there's an error setting up the routing manager, display the error message
            print(f"error setting up routingindexmanager: {e}")
            return
        
        #create a route 'model' using the data we input
        routing = pywrapcp.RoutingModel(manager)
        
        #this function searches through the data model and figures out how much time it takes to go to/from two points
        def time_callback(from_index, to_index):
            #from_node and to_node are two locations we are comparing travel to on the map
            #we are converting from 'index' location on the list to 'dictionary' items in the data structure
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            #return the travel time from the time matrix
            return data['time_matrix'][from_node][to_node] 
        
        #tell the routing optimizer that we want to use the function above to figure out time between locations
        transit_callback_index = routing.RegisterTransitCallback(time_callback)
        #and now, we tell the routing optimizer to calculate 'cost' based on the time between locations
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        #there are different settings to try and find the fastest route, we pick default settings
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        #we say the 'solution' is the route with the cheapest 'arc' aka the least amount of time
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )
        #set a time limit for the optimization process
        search_parameters.time_limit.seconds = 30 
        
        print("optimizing route...")  
        #now, we tell the code to solve the route optimization all by itself
        solution = routing.SolveWithParameters(search_parameters)
        
        if solution:
            #if a solution is sucessfully found
            print("route optimization successful.")
            #figure out where the route starts (it should be our home location)
            index = routing.Start(0)
            #store the stops of the route in a list
            route = []
            while not routing.IsEnd(index):
                #figure out what 'node' aka location we are looking at
                node_index = manager.IndexToNode(index)
                route.append(node_index)
                #check where the next 'node' is, then repeat to add it to our list
                index = solution.Value(routing.NextVar(index))
            #if we are returning back to home, add home as the last location to visit
            if return_to_start_checkbox.value:
                #the route should return to the starting location
                node_index = manager.IndexToNode(index)
                #add the final home location to the list of locations
                route.append(node_index)
            
            #now, we are doing a conversion from the weird route optimizer data structure back to regular location ID's
            route_order = {location_idx: order for order, location_idx in enumerate(route)}
            
            #keep track of colors for the markers
            location_colors = {}
            
            #now, we are going to update all the markers on the map to be colored with the opimal route
            for loc_idx, marker in enumerate(location_markers):
                #if the location is in our optimal route (it should be)
                if loc_idx in route_order:
                    #figure out which 'stop' along our route it is
                    order = route_order[loc_idx]
                    #if it's our first location, aka home, we make it the pink color with home icon
                    if loc_idx == 0:
                        #if it's the first location (home), set a pink home icon
                        icon = AwesomeIcon(name='home', marker_color='pink', icon_color='white')
                        location_colors[loc_idx] = 'pink'
                        #when we click on the icon, it displays 'home'
                        marker.popup = widgets.HTML("home")
                    else:
                        #for all other locations, assign colors from the list of colors we picked up at the start of the program
                        #figure out which 'index' the color is aka which stop number the location is at
                        #if we have more stops than the length of the list, the % len... will loop back thru the list again
                        color_index = (order - 1) % len(marker_colors)
                        #figure out what color we are using
                        color = marker_colors[color_index]
                        #store the color for later refrence
                        location_colors[loc_idx] = color
                        #update the marker with the new color and shopping bag icon
                        icon = AwesomeIcon(name='shopping-bag', marker_color=color, icon_color='white')
                        marker.popup = widgets.HTML(f"marker color: {color.capitalize()}")
                    marker.icon = icon  # update the marker's icon on the map
                else:
                    #if the location is not part of the route, keep the original icon
                    pass
            
            #now, we are getting directions between each location on the route using API calls
            num_direction_calls = len(route) - 1
            #estimate that each API call will take 1.5 seconds
            time_estimate_directions = int(num_direction_calls * 1.5)
            print(f"fetching directions for the optimized route ({num_direction_calls} api calls, approximately {time_estimate_directions} seconds)...")
            
            #store each 'leg' of travel between each location
            legs = []
            #keep track of overall time and overall distance travelling
            overall_travel_time = 0
            overall_travel_distance = 0
            #for each 'leg' segment, we get the info needed to make directions API call
            for i in range(len(route) - 1):
                #the location to travel from
                #this finds the index on the list, so we can get the latitude and longitude soon
                from_idx = route[i]
                #the location to travel to
                to_idx = route[i + 1]
                #if the first or last location is 'home' then we keep track that we are travelling home for reporting
                from_location = "home" if from_idx == 0 else f"{location_colors.get(from_idx, 'unknown').capitalize()} marker"
                to_location = "home" if to_idx == 0 else f"{location_colors.get(to_idx, 'unknown').capitalize()} marker"
                #get the coordinates of the to and from travel locations
                coords = [
                    (locations[from_idx]['lng'], locations[from_idx]['lat']),  
                    (locations[to_idx]['lng'], locations[to_idx]['lat']) 
                ]
                try:
                    #prepare everything for the API call
                    params = {
                        'coordinates': coords,             #set the start and end coordinates
                        'profile': 'driving-car',         #set the mode of transportation to driving
                        'format_out': 'geojson',          #set the output format to geojson
                        'preference': 'fastest',          #prefer the fastest route
                        'geometry': 'true',               #include the geometry of the route
                        'instructions': 'true',           #include step-by-step instructions
                        'options': {
                            'avoid_polygons': mapping(avoid_polygon)  #include avoid areas if any
                        } if avoid_polygon else {}
                    }
                    #make the API call to get directions for this leg
                    route_result = ors_client.directions(**params)
                    #get the polyline geometry coordinates so we can display it on the map
                    route_geometry = route_result['features'][0]['geometry']
                    
                    #put the coordinates in the correct order
                    line_coords = [(lat, lng) for lng, lat in route_geometry['coordinates']]
                    
                    #create a polyline object to represent the route segment on the map
                    segment_polyline = Polyline(
                        locations=line_coords,          #show the line of the polyline
                        color='#c71585',               #set the color of the polyline (dark pink)
                        weight=4,                       #set the thickness of the polyline
                        opacity=0.8,                    #set the transparency of the polyline
                        fill=False                      #ensure the polyline is not filled
                    )
                    route_layer.add_layer(segment_polyline)  #add the polyline to the route layer on the map
                    
                    #extract the segments of the route to get detailed directions
                    segments = route_result['features'][0]['properties']['segments']
                    #store steps for this leg
                    leg_steps = []  
                    #total travel time for this leg in seconds
                    leg_travel_time = 0 
                    #total travel distance for this leg in meters
                    leg_travel_distance = 0
                    #for each line segment, we are going to 
                    for segment in segments:
                        #iterate through each segment in the route
                        for step in segment['steps']:
                            #iterate through each step in the segment
                            instruction = step['instruction']  #get the instruction for this step
                            distance = step['distance']          #get the distance for this step in meters
                            duration = step['duration']          #get the duration for this step in seconds
                            #add the step details to the leg_steps list
                            leg_steps.append({
                                'instruction': instruction,
                                'distance': distance,
                                'duration': duration
                            })
                            leg_travel_time += duration           #add in the duration
                            leg_travel_distance += distance       #add in the distance
                    #add the leg details to the legs list
                    legs.append({
                        'from': from_location,                   #starting location of the leg
                        'to': to_location,                       #ending location of the leg
                        'steps': leg_steps,                      #list of steps in the leg
                        'travel_time': leg_travel_time,          #total travel time for the leg
                        'travel_distance': leg_travel_distance   #total travel distance for the leg
                    })
                    #add to the overall travel time
                    overall_travel_time += leg_travel_time     
                    #add to the overall travel distance
                    overall_travel_distance += leg_travel_distance  
                except Exception as e:
                    #if there's an error fetching directions for this leg, display the error message
                    print(f"error fetching route for leg from {from_location} to {to_location}: {e}")
                time.sleep(1)  #wait for 1 second before calling API again so we don't overload limits

            #if there is information to display, print it all out
            if legs:
                #if there are any legs to display directions for
                print("\ndirections for the optimized route:")  #say that directions will be displayed
                for idx, leg in enumerate(legs, start=1):
                    #iterate through each leg and display its details
                    from_location = leg['from']  #get the starting location of the leg
                    to_location = leg['to']      #get the ending location of the leg
                    print(f"\nleg {idx}: {from_location} → {to_location}")  #print the leg number and locations
                    for step_idx, step in enumerate(leg['steps'], start=1):
                        #iterate through each step in the leg and display instructions
                        instr = step['instruction']          #get the instruction for the step
                        dist_m = step['distance']            #get the distance for the step in meters
                        dur_s = step['duration']             #get the duration for the step in seconds
                        dist_miles = dist_m * MILES_PER_METER  #convert distance to miles
                        dur_min = dur_s / 60                  #convert duration to minutes
                        #print the step details with instruction, distance, and duration
                        print(f"{step_idx}. {instr} ({dist_miles:.2f} miles, {dur_min:.1f} minutes)")
                    #calculate and display the total travel time and distance for the leg
                    leg_travel_time_min = leg['travel_time'] / 60  #convert total leg time to minutes
                    leg_travel_distance_miles = leg['travel_distance'] * MILES_PER_METER  #convert total leg distance to miles
                    print(f"total travel time for leg {idx}: {leg_travel_time_min:.1f} minutes")  #display total time
                    print(f"total travel distance for leg {idx}: {leg_travel_distance_miles:.2f} miles")  #display total distance
                
                #calculate and display the overall travel time and distance for the entire route
                overall_travel_time_min = overall_travel_time / 60  #convert total travel time to minutes
                overall_travel_distance_miles = overall_travel_distance * MILES_PER_METER  #convert total travel distance to miles
                print(f"\noverall travel time: {overall_travel_time_min:.1f} minutes")        #display overall travel time
                print(f"overall travel distance: {overall_travel_distance_miles:.2f} miles")  #display overall travel distance
            else:
                #if no legs were found, inform the user that no directions are available
                print("no directions available.")
            
            print("route optimization completed. the map has been updated.")  #inform the user that optimization is complete
        else:
            #if no solution was found for the routing problem, inform the user
            print('no solution found!')


#when we click the optimize button, do everything defined above
optimize_button.on_click(optimize_route)

#function to create a time matrix considering areas to avoid
def get_time_matrix_with_avoid(locations, ors_client, avoid_polygon=None):
    num_locations = len(locations)  #get the total number of locations
    #initialize a matrix filled with zeros, size num_locations x num_locations
    time_matrix = [[0]*num_locations for _ in range(num_locations)]
    for i in range(num_locations):
        for j in range(num_locations):
            if i == j:
                #if it's the same location, travel time is zero
                time_matrix[i][j] = 0
                continue
            coords = [
                (locations[i]['lng'], locations[i]['lat']),  #longitude and latitude of the starting location
                (locations[j]['lng'], locations[j]['lat'])   #longitude and latitude of the ending location
            ]
            try:
                #prepare parameters for the directions API call
                params = {
                    'coordinates': coords,             #set the start and end coordinates
                    'profile': 'driving-car',          #set the mode of transportation to driving
                    'format_out': 'geojson',           #set the output format to geojson
                    'preference': 'fastest',           #prefer the fastest route
                    'geometry': 'true',                #include the geometry of the route
                    'instructions': 'false',           #no need for step-by-step instructions in this piece
                    'options': {
                        'avoid_polygons': mapping(avoid_polygon)  #include avoid areas if any
                    } if avoid_polygon else {}
                }
                #make the API call to get directions
                route = ors_client.directions(**params)
                #extract the duration (travel time) from the API response
                duration = route['features'][0]['properties']['summary']['duration']
                #store the duration in seconds
                time_matrix[i][j] = int(duration)  
            except Exception as e:
                #if there's an error fetching the time, display the error message and set time to infinity
                print(f"error fetching time from {locations[i]['name']} to {locations[j]['name']}: {e}")
                time_matrix[i][j] = float('inf')
            #wait for 1 second to not overload API
            time.sleep(1)  
    return time_matrix

#function to create the data model required for the routing problem
#this is basically a way to standardize the data and store it in one locaiton (like a spreadsheet but code)
def create_data_model(time_matrix):
    #initialize an empty dictionary to store data
    data = {}  
    #store the time matrix in the data dictionary
    data['time_matrix'] = time_matrix  
    #set the number of vehicles to 1 (single route)
    data['num_vehicles'] = 1            
    #set the depot (starting point) as the first location
    data['depot'] = 0                    
    return data

#display the optimize button and the output area on the user interface
#show the optimize route button
display(optimize_button)  
#show the output section where results will be displayed
display(output)           


Map(center=[44.96, -93.18], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

Button(button_style='success', description='Optimize Route', style=ButtonStyle())

Output()